In [262]:
import re, nltk
import cleantext as ct
import pandas as pd
from itertools import chain
from nltk.corpus import stopwords
from nltk import regexp_tokenize
from sklearn.model_selection import train_test_split

In [263]:
def tokenize(string):
    return regexp_tokenize(string, pattern=r"[^\w\d\-''_<>]", gaps=True)

In [264]:
def stem(lst):
  stemmer = nltk.SnowballStemmer(language="english", ignore_stopwords=True)
  stem_words = []
  for token in lst:
    x = stemmer.stem(token)
    stem_words.append(x)
  return stem_words

In [265]:
def remove_stopwords(lst):
  for token in lst:
    stop_words = stopwords.words('english')
    filteredInput = []
    
    for token in lst:
      if token not in stop_words:
        filteredInput.append(token)
        
    return filteredInput

In [266]:
def cleaning(csv):
  data = csv
  df = pd.read_csv(data)
  # counter = 0
  
  for i in range(len(df["content"])):
    # # rename fake-like types to just fake for simplicity
    # if df.loc[i, 'type'] in ["unreliable", "fake", "conspiracy", "junksci"]:
    #   df.loc[i, 'type'] = "fake"
    # elif df.loc[i, 'type'] != 'reliable':
    #   df = df.drop(i)
    #   continue
    value = df.loc[i, "content"]
    value = ct.clean(value, lower=True, no_line_breaks=True, no_numbers=True, no_emails=True, no_urls=True, replace_with_number="<NUM>")
    value = tokenize(value)
    value = remove_stopwords(value)
    value = stem(value)
    df.loc[i, "content"] = ' '.join(map(str,value))
  # df.reset_index(drop=True)
  return df

In [267]:
def run():
  df = cleaning("news_sample.csv")
  df.to_csv("cleaned_news_sample.csv")
run()

## Libraries Brugt
### pandas
Converter csv til DataFrame, som kan bruges til at manipulere dataen inde i csv filen nemt
### nltk
Har indbygget pakker til at tokenize text og fjerne stopwords
### cleantext
Renser texten

In [268]:
def analyze_data(csv):
  df = pd.read_csv(csv)
  
  urlsInReliable = 0
  urlsInFake = 0
  datesInFake = 0
  datesInReliable = 0
  numInFake = 0
  numInReliable = 0
  isFake = 0
  isReliable = 0
  
  df = df.filter(items=["type", "content"])

  for col, row in df.iterrows():
    isFake = 0
    isReliable = 0
    if (row["type"] == "fake"):
      isFake += 1
    else:
      isReliable += 1
    
    for word in row["content"].split():
      urlsInFake += int("<url>" == word) * isFake
      urlsInReliable += int("<url>" == word) * isReliable
      
      datesInFake += int("<date>" == word) * isFake
      datesInReliable += int("<date>" == word) * isReliable
      
      numInFake += int("<num>" == word) * isFake
      numInReliable += int("<num>" == word) * isReliable
    
    
    dictionary = dict()
    pattern = re.compile(r"\w+")
    lst = re.findall(pattern, row["content"])
    
    for word in lst:
      if not dictionary.get(word):
        dictionary[word] = 1
      else:
        dictionary[word] += 1
    
    dictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    dictionary = dictionary[0:100]

  
  return urlsInFake, urlsInReliable, datesInFake, datesInReliable, numInFake, numInReliable, dictionary
    
analyze_data("cleaned_news_sample.csv")

(239,
 4,
 0,
 0,
 1420,
 726,
 [('reuter', 3),
  ('journalist', 3),
  ('myanmar', 3),
  ('clinton', 2),
  ('immedi', 2),
  ('releas', 2),
  ('held', 2),
  ('free', 2),
  ('num', 2),
  ('appear', 2),
  ('court', 2),
  ('former', 1),
  ('u', 1),
  ('presid', 1),
  ('bill', 1),
  ('monday', 1),
  ('call', 1),
  ('two', 1),
  ('press', 1),
  ('critic', 1),
  ('societi', 1),
  ('detent', 1),
  ('anywher', 1),
  ('unaccept', 1),
  ('said', 1),
  ('twitter', 1),
  ('post', 1),
  ('accus', 1),
  ('report', 1),
  ('wa', 1),
  ('lone', 1),
  ('kyaw', 1),
  ('soe', 1),
  ('oo', 1),
  ('breach', 1),
  ('countri', 1),
  ('offici', 1),
  ('secret', 1),
  ('act', 1),
  ('little', 1),
  ('us', 1),
  ('law', 1),
  ('coloni', 1),
  ('rule', 1),
  ('due', 1),
  ('main', 1),
  ('citi', 1),
  ('yangon', 1),
  ('wednesday', 1),
  ('second', 1),
  ('prosecutor', 1),
  ('could', 1),
  ('request', 1),
  ('charg', 1),
  ('file', 1)])

**counting the number of URLs in the content**
- svar her

**counting the number of dates in the content**
- svar her

**counting the number of numeric values in the content**
- svar her

**determining the 100 more frequent words that appear in the content**
- svar her, mangler kode

**plot the frequency of the 10000 most frequent words (any interesting patterns?)**
- svar her, mangler kode

**run the analysis in point 4 and 5 both before and after removing stopwords and applying stemming: do you see any difference?**
- svar her, mangler kode


**Make at least three non-trivial observations/discoveries about the data. These observations could be related to outliers, artefacts, or even better: genuinely interesting patterns in the data that could potentially be used for fake-news detection.**
- Indeholder subjektivt ladede ord, så som "stupid"
- Mangler kilder
- Stavefejl

**Describe how you ended up representing the FakeNewsCorpus dataset (for instance with a Pandas dataframe). Argue for why you chose this design.**
- For at repræsentere data har vi valgt Pandas dataFrame. Det har vi gjort fordi det gør det nemt og overskueligt at arbejde med dataen og rense og manipulere med den.

**Did you discover any inherent problems with the data while working with it?**
- Ikke rigtigt

**Report key properties of the data set - for instance through statistics or visualization.**

In [269]:
# Her splitter vi data

Task 0: Vi valgte at gruppere kategorierne ind i "fake" og 